In [ ]:
from pprint import pprint
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms, utils
from torch.utils import data

import matplotlib.pyplot as plt
import numpy as np
from tqdm.auto import trange

> 강의이후 라이브러리 사용중 막히는 것이 있을때는,  
Pytorch Official Documentation: https://pytorch.org/docs/stable/index.html  
을 참고하시는 것을 권장드립니다

In [ ]:
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")
print(DEVICE)

In [ ]:
EPOCHS = 20
BATCH_SIZE = 64

In [ ]:
transform = transforms.Compose([
    transforms.ToTensor()
])

In [ ]:
trainset = datasets.FashionMNIST(
    root      = './data/', 
    train     = True,
    download  = True,
    transform = transform
)

testset = datasets.FashionMNIST(
    root      = './data/', 
    train     = False,
    download  = True,
    transform = transform
)

In [ ]:
train_loader = data.DataLoader(dataset=trainset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)
test_loader = data.DataLoader(dataset=testset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)

In [ ]:
dataiterator = iter(train_loader)
images, labels = next(dataiterator)

In [ ]:
images.shape

In [ ]:
labels

In [ ]:
img   = utils.make_grid(images, padding=0)
# 자동으로 RGB로 바뀜
print(img.shape)
# matplotlib은 이미지를 가로x세로x채널로 받으므로 차원을 한칸씩 밀어준다
img = img.permute((1,2,0))
print(img.shape)
plt.figure(figsize=(7, 7))
plt.imshow(img)
plt.show()

In [ ]:
CLASSES = {v: k for k, v in trainset.class_to_idx.items()}
pprint(CLASSES)

In [ ]:
for i, label in enumerate(labels):
    index = label.item()
    print("{:12}".format(CLASSES[index]), end=' ')
    if i % 8 == 7:
        print()

In [ ]:
idx = 0
item_img = images[idx]
item_npimg = item_img.squeeze().numpy()
plt.title(CLASSES[labels[idx].item()])
plt.imshow(item_npimg, cmap='gray')
plt.show()

## 아래 네트워크를 완성해보세요.
* Hint
1. 이미지는 2차원, Linear레이어는 1차원 벡터를 입력으로 받습니다
2. view함수를 활용하세요
#### 활용이 가능하신 분은 Baseline보다 더 나은 성능을 내보세요.
### Baseline: `20 epoch` train 후, test accuracy `85.59 %`

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        '''
        code here
        '''
    
    def forward(self, x):
        '''
        code here
        '''
        return x

In [ ]:
model = Net().to(DEVICE)
optimizer = optim.SGD(model.parameters(), lr=0.01)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma = 0.1)

In [ ]:
def train(model, train_loader, optimizer):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data = data.view(-1, 28 * 28)
        data, target = data.to(DEVICE), target.to(DEVICE)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()

In [ ]:
def evaluate(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data = data.view(-1, 28 * 28)
            data, target = data.to(DEVICE), target.to(DEVICE)
            output = model(data)
            test_loss += F.cross_entropy(output, target,
                                         reduction='sum').item()
            pred = output.max(dim=1)[1]
            correct += pred.eq(target).sum().item()

    test_loss /= len(test_loader.dataset)
    test_accuracy = 100. * correct / len(test_loader.dataset)
    return test_loss, test_accuracy

In [ ]:
test_loss, test_accuracy = evaluate(model, test_loader)
print(f'[{0:3}] Test Loss: {test_loss:.4f}, Accuracy: {test_accuracy:.2f} %')
for epoch in trange(1, EPOCHS + 1):
    train(model, train_loader, optimizer)
    test_loss, test_accuracy = evaluate(model, test_loader)
    print(f'[{epoch:3}] Test Loss: {test_loss:.4f}, Accuracy: {test_accuracy:.2f} %')

In [ ]:
columns = 6
rows = 6
fig = plt.figure(figsize=(10,10))
 
model.eval()
with torch.no_grad():
    for i in range(1, columns*rows+1):
        data_idx = np.random.randint(len(testset))
        input_img = testset[data_idx][0].unsqueeze(dim=0).view(-1, 28 * 28).to(DEVICE)
        output = model(input_img)
        _, argmax = torch.max(output, 1)
        pred = CLASSES[argmax.item()]
        label = CLASSES[testset[data_idx][1]]
        
        fig.add_subplot(rows, columns, i)
        if pred == label:
            plt.title(label+'/'+pred)
            cmap = 'Blues'
        else:
            plt.title(label+'/'+pred)
            cmap = 'Reds'
        plot_img = testset[data_idx][0][0,:,:]
        plt.imshow(plot_img, cmap=cmap)
        plt.axis('off')
    
plt.show() 

## MNIST dataset에 대해서도 적용해봅시다!

In [ ]:
trainset = datasets.MNIST(
    root      = './.data/', 
    train     = True,
    download  = True,
    transform = transform
)

testset = datasets.MNIST(
    root      = './.data/', 
    train     = False,
    download  = True,
    transform = transform
)

In [ ]:
CLASSES = {v: k.split(' - ')[1] for k, v in trainset.class_to_idx.items()}
pprint(CLASSES)

## train 코드 작성

1. 데이터 로더 생성
2. 모델 생성
3. optimizer, lr scheduler 생성
4. 정해진 epoch 만큼 train 반복

In [ ]:
'''
code here
'''

## 학습결과 평가

In [ ]:
columns = 6
rows = 6
fig = plt.figure(figsize=(10,10))
 
model.eval()
for i in range(1, columns*rows+1):
    data_idx = np.random.randint(len(testset))
    input_img = testset[data_idx][0].unsqueeze(dim=0).view(-1, 28 * 28).to(DEVICE) 
    
    output = model(input_img)
    _, argmax = torch.max(output, 1)
    pred = CLASSES[argmax.item()]
    label = CLASSES[testset[data_idx][1]]
    
    fig.add_subplot(rows, columns, i)
    if pred == label:
        plt.title(label+'/'+pred)
        cmap = 'Blues'
    else:
        plt.title(label+'/'+pred)
        cmap = 'Reds'
    plot_img = testset[data_idx][0][0,:,:]
    plt.imshow(plot_img, cmap=cmap)
    plt.axis('off')
    
plt.show() 